Будем использовать реализацию Word2vec в библиотеке **Gensim**, а в качестве предобученных моделей возьмем модели Андрея Кутузова и Лизы Кузьменко с сайта [RusVectōrēs.](https://rusvectores.org/ru/models/). 

pip install gensim

In [1]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors
import gensim
gensim.__version__
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


В качестве моделей давайте возьмем 

1) araneum_none_fasttextcbow_300_5_2018 (fasttext) - модель, обученная на интернет-корпусе русского языка


2) ruscorpora_upos_skipgram_300_5_2018 (word2vec) - модель, обученная НКРЯ

## word2vec + fasttext

Существуют несколько форматов, в которых могут храниться модели - .vec и .model 

1) Первый формат считается классическим вариантом модели word2vec. Для загрузки таакой модели надо воспользоваться методом *KeyedVectors.load_word2vec_format*. 
Модель может быть бинарной, для ее загрузки надо передать параметр binary, равный True. 

2) Формат .model - собственный формат gensim. Такую модель надо загружать с помощью метода *KeyedVectors.load*.

 **1) если модель без тэгов**

In [2]:
!wget -c https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2020-10-20 18:56:51--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [3]:
!tar --gunzip --extract --verbose --file=araneum_none_fasttextcbow_300_5_2018.tgz

araneum_none_fasttextcbow_300_5_2018.model
araneum_none_fasttextcbow_300_5_2018.model.vectors_ngrams.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors_vocab.npy


In [4]:
!wget -c https://rusvectores.org/static/models/rusvectores4/RNC/ruscorpora_upos_skipgram_300_5_2018.vec.gz

--2020-10-20 18:57:47--  https://rusvectores.org/static/models/rusvectores4/RNC/ruscorpora_upos_skipgram_300_5_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [5]:
!tar --gunzip --extract --verbose --file=ruscorpora_upos_skipgram_300_5_2018.vec.gz
#import tarfile
#tar = tarfile.open('ruscorpora_upos_skipgram_300_5_2018.vec.gz')
#tar.extractall()

#import gzip, shutil

#with gzip.open('ruscorpora_upos_skipgram_300_5_2018.vec.gz', 'r') as f_in, open('ruscorpora_upos_skipgram_300_5_2018.vec', 'wb') as f_out:
#  shutil.copyfileobj(f_in, f_out)

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Archive contains ‘ 0.102193 -0’ where numeric off_t value expected
tar: Exiting with failure status due to previous errors


In [6]:
# загрузка модели
#import tarfile

model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)


#проверка наличия слова в словаре

lemma = 'заграница'
lemma in model

True

**2) если модель с POS-тэггингом**

In [8]:
# загрузка модели


#model_file = 'ruscorpora_upos_skipgram_300_5_2018.vec'

#model_POS = KeyedVectors.load_word2vec_format(model_file, binary=False)


#проверка наличия слова в словаре

#lemma = 'заграница_NOUN'
#lemma in model_POS

In [9]:
#model_POS

**3) получение вектора слова**

In [10]:
#model['заграница']

In [11]:
#model_POS['заграница_NOUN']

In [12]:
lemma = 'заграница'
lemma in model

True

In [13]:
#v1 = model['заграница']
#v2 = model_POS.wv['заграница_NOUN']

#(v1 == v2).all()

## Получение вектора документа

Отлично, вектора для слов получены. Что с ними делать дальше? 

Есть два подхода (а точнее есть один, а второй мы придумали, потому что с одним жить нельзя).
> Классика - для получения вектора документа нужно взять и усреднить все вектора его слов
 
$$ vec_{doc} = \frac {\sum_{i=0}^{n} vec_i}{len(d)} $$


In [14]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный', 'замок', 'стоял', 'на', 'утёсе']

# создаем вектор-маску
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model:
        lemmas_vectors[idx] = model[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)


In [15]:
lemmas_vectors

array([[-0.00911347, -0.02530465,  0.05348665, ..., -0.05241832,
         0.01358955,  0.00458029],
       [-0.05764473, -0.00463477,  0.11919285, ..., -0.02958015,
         0.00869342,  0.03035057],
       [ 0.02615208,  0.01726465,  0.00357048, ...,  0.00672541,
        -0.00778436, -0.02008392],
       [ 0.01678883, -0.00648571, -0.00549603, ..., -0.0208534 ,
         0.00960696,  0.01206898],
       [ 0.01002831,  0.01089021,  0.01548253, ...,  0.0052027 ,
         0.02089224, -0.01940127]])

> Эксперимент - представим документ не в виде одного уредненного вектора, а как матрицу векторов входящих в него слов

```
 слово1 |  v1_300
 слово2 |  v2_300
 слово3 |  v3_300
 слово4 |  v4_300
```

> Отлично, теперь каждый документ представлен в виде матрицы векторов своих слов. Но нам надо получить близость матрицы документа в коллекции и матрицы входящего запроса. Как? Умножим две матрицы друг на друга - одна матрица размером d x 300, другая q x 300 - получим попарную близость слов из каждого документа - матрицу размером d x q.


In [16]:
# возьмем игрушечный пример кейса

text1 = 'турция' 
text2 = 'нужна справка срочно'
query = 'быстрая справка'

In [17]:
def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

In [18]:
# построим матрицы всех документов

def create_doc_matrix(text):
    lemmas = text.strip().split(' ')
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))
    for idx, lemma in enumerate(lemmas):
        print(lemma)
        #print(model.wv)
        if lemma in model.wv:
            lemmas_vectors[idx] = normalize_vec(model.wv[lemma])
            
    return lemmas_vectors    


text1_m = create_doc_matrix(text1)
text2_m = create_doc_matrix(text2)
query_m = create_doc_matrix(query)

турция
нужна
справка
срочно
быстрая
справка


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


In [19]:
# размер матрицы как и ожидали
query_m.shape
#query_m

(2, 300)

In [20]:
# посмотрим на близость слов первого текста и слов запроса
text1_m.dot(query_m.T)

array([[0.09587915, 0.01183069]])

In [21]:
# посмотрим на близость слов второго текста и слов запроса
text2_m.dot(query_m.T)

array([[-0.0260624 ,  0.11607588],
       [ 0.01341236,  1.00000011],
       [ 0.22505549,  0.33582122]])

In [22]:
docs_m = [text1_m, text2_m]

    
def search(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    print(sims)
    return np.argmax(sims)


search(docs_m, query_m)

[0.10770983955697251, 1.225055597288777]


1

## Задание

Реализуйте поиск по нашему стандартному Covid корпусу с помощью модели на Araneum двумя способами:

    1. преобразуйте каждый документ в вектор через усреднение векторов его слов и реализуйте поисковик как 
    обычно через умножение матрицы документов коллекции на вектор запроса 
    2. экспериментальный способ - реализуйте поиск ближайшего документа в коллекции к запросу, преобразовав 
    каждый документ в матрицу (количество слов x размер модели)
    
Посчитайте качество поиска для каждой модели на тех же данных, что и в предыдущем задании. В качестве препроцессинга используйте две версии - с удалением NER и без удаления.  

In [23]:
## Способ 1

from sklearn.feature_extraction.text import TfidfVectorizer

import re
import pandas as pd
import unicodedata
from sklearn.model_selection import train_test_split

# инициализируем
vectorizer = TfidfVectorizer()

answers = pd.read_excel('answers_base.xlsx')
queries = pd.read_excel('queries_base.xlsx')

train, test = train_test_split(queries, test_size=0.3)

new_train = pd.DataFrame()
new_train['Текст вопроса'] = pd.concat([train['Текст вопроса'],answers['Текст вопросов']],ignore_index = True)
new_train['Номер связки\n'] = pd.concat([train['Номер связки\n'], answers['Номер связки']], ignore_index=True)

from sklearn.feature_extraction.text import TfidfVectorizer

count_vect = TfidfVectorizer()

X_train = count_vect.fit_transform(new_train['Текст вопроса'].values.astype('U'))
y_train = new_train['Номер связки\n']

X_test = count_vect.transform(test['Текст вопроса'].values.astype('U'))
y_test = test['Номер связки\n']

Y_t = X_test.toarray().transpose()

In [24]:
#lis = 'Мама мыла раму.'
#lis.split(' ')
X_test.dot(Y_t)

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.06797762],
       [0.        , 1.        , 0.04273532, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04273532, 1.        , ..., 0.00144608, 0.03872271,
        0.06194492],
       ...,
       [0.        , 0.        , 0.00144608, ..., 1.        , 0.08094187,
        0.00443409],
       [0.        , 0.        , 0.03872271, ..., 0.08094187, 1.        ,
        0.01517981],
       [0.06797762, 0.        , 0.06194492, ..., 0.00443409, 0.01517981,
        1.        ]])

In [25]:
#
#import pymorphy2
#morph = pymorphy2.MorphAnalyzer()

ModuleNotFoundError: ignored

In [ ]:
#morph.parse('Котики')[0][2]

# Способ 1: с ner

In [28]:
#import numpy as np

list_of_lemmas = []
for t in new_train['Текст вопроса']:
  #new_list = []
  t = re.sub('[!@#$%^&*\(\)\[\]\{\}.,><?/-/"]',' ',str(t))
  t = re.sub('\n', ' ', str(t))
  #list_of_lemmas.append(t)
  #for word in t:
  #  new_list.append(morph.parse(t)[0][2])
  list_of_lemmas.append(''.join(t))

In [29]:
list_of_lemmas

['Я вернулась из поездки в ДНР-Донецкая Народная Республика в которой находилась с 19 08 20 по 30 08 20  В ДНР у меня проживает мать которой проводилась медицинская операция замена хрусталика  и мое присутствие было там необходимо  Пересекала границу я на личном автотранспорте в пункте пропуска Новоазовск Ростовской области  Необходимо ли мне сейчас сдавать тест на covid-19  Если да- то в какие сроки \xa0и \xa0где сдавать\xa0анализы  С больными людьми не контактировала признаков ОРВИ нет   ',
 'Следует ли из всего написанного ниже  что в течении 3х дней прибывающие на\xa0 территорию РФ обязаны именно СДАТЬ тест  а загрузить результаты можно и позже  после их получения Отправлено со смартфона Samsung Galaxy ',
 ' 30 07 20 сдала анализы в\xa0 ф-ле 64 пол-ки по ул 2-я Пугачевская на антитела   результат-оказалась носителем короновируса И тишина  Никто из медперсонала меня не наблюдал  не справлялся о моем состоянии здоровья  как будто ничего не произошло \xa0 Сегодня получила результаты а

In [ ]:
vecs = []

for lemmas in list_of_lemmas:
  lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
  vec = np.zeros((model.vector_size,))
  for idx, lemma in enumerate(lemmas):
    if lemma in model:
      lemmas_vectors[idx] = model[lemma]
        
  if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)
  vecs.append(vec)

In [ ]:
#vecs[0]
#vecs

#count_vect.fit_transform(

In [ ]:
len(new_train)

In [ ]:
len(vecs)

In [ ]:
#list_of_lemmas

In [ ]:
ner_vecs = new_train

In [ ]:
ner_vecs['Вектор'] = vecs

In [ ]:
ner_vecs

In [ ]:
queries['Текст вопроса']

vec_2 = count_vect.fit_transform(ner_vecs['Вектор'].fillna(0).astype(str))
#X = vec_2

#for v in ner_vecs['Вектор']:
  #print(v)
#  matr_list.append(X_train.dot(v))

#ner_vecs['Умножение матрицы на вектор'] = 

In [ ]:
y = count_vect.transform(ner_vecs['Вектор'].astype(str))

In [ ]:
#vec_2_t = vec_2.transpose()
type(vec_2)

In [ ]:
#vec_2_t.dot(y)

#vec_2.dot(y.transpose())

mat_vec = []

for v in ner_vecs['Вектор']:
  #print(v)
  v_t = v.transpose()
  mat_vec.append(v)

In [ ]:
mat_vec

In [ ]:
#!pip install natasha

#from natasha import (
#    Segmenter,
#    
#    NewsEmbedding,
#    NewsMorphTagger,
#    NewsSyntaxParser,
#    
#    Doc
#)

#def preprocess_with_natasha(text: str) -> str:
#  segmenter = Segmenter()
#  emb = NewsEmbedding()
#  morph_tagger = NewsMorphTagger(emb)
#  syntax_parser = NewsSyntaxParser(emb)

#  doc = Doc(text)

#  doc.segment(segmenter)
#  doc.tag_morph(morph_tagger)
#  doc.parse_syntax(syntax_parser)

#  list_of_needed = {}

#  sent = doc.sents[0]
#  for ss in sent.morph:
#    for s in ss:
#      search1 = re.findall('text=\'(.*?)\'',str(s))
#      search2 = re.findall('pos=\'(.*?)\'',str(s))
#      list_of_needed[' '.join(search1)] = ' '.join(search2)

#  for k,v in list_of_needed.items():
#    if v == 'PROPN':
#      text = re.sub(k, '', text)
#  return text

from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

!pip install git+https://github.com/deepmipt/ner
import ner
extractor = ner.Extractor()

def preprocess_with_deepmipt(text: str) -> str:
  list_of_extractors = []
  for m in extractor(text):
    for mm in m:
      if str(type(mm)) == '<class \'list\'>':
        list_of_extractors.append(mm)
  list_2 = []
  for l in list_of_extractors:
    word = re.findall('text\=\'(.*?)\'', str(l))
    if type(word) is list:
      list_2 = list_2 + word
    else:
      list_2.append(' '.join(word))
  for word in text.split():
    for ww in list_2:
      if word == ww:
        text = re.sub(word, '', text)
  return text


In [ ]:
list_of_words = []
for d in new_train['Текст вопроса'].fillna('...'):
  #print(type(d))
  list_of_words.append(preprocess_with_deepmipt(str(d)))


In [ ]:
list_of_words
#preprocess_with_natasha('Они были там, где была Москва.')

In [ ]:
#X_train = vectorizer.fit_transform(sevvo_4['Текст вопроса'].values.astype('U'))
#y_train = sevvo_4['Номер связки'].values.astype('U')

In [ ]:
#X_test = vectorizer.transform(other_seventy['Текст вопроса'].values.astype('U'))
#y_test = other_seventy['Номер связки\n'].values.astype('U')

In [ ]:
#print(X_train.shape)
#print(X_test.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *

clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)

predicted


#from sklearn.tree import DecisionTreeClassifier
#clf = DecisionTreeClassifier().fit(X_train, y_train)
#predicted = clf.predict(X_test)

#acc = accuracy_score(y_test, predicted)
#print('acc={0:1.4f}'.format(acc))

In [ ]:
#import operator

#def ranging(text):
#  dict_texts = {}
#  cnt = 0
#  for x in np.nditer(text):
#    dict_texts[n_conn[cnt]] = float(x)
#    cnt += 1
#  new_dict = sorted(dict_texts.items(),
#                    key = operator.itemgetter(1), reverse = True)
#  first = []
#  for k in new_dict:
#    first.append(k[0])
#  return first[0]

#ranging_list = []

#for o in X.dot(Y_t):
#  ranging_list.append(ranging(o))

#other_seventy['tf_none'] = ranging_list[:690]

#accuracy = 0
#for s, t in zip(other_seventy['Номер связки\n'],other_seventy['tf_none']):
#  if s == t:
#    accuracy += 1

#acc = accuracy/len(other_seventy['Номер связки\n'])
#print(acc)

#from sklearn.metrics import accuracy_score
#print(accuracy_score(sevvo_4['Номер связки'].notnull(), ranging_list))

In [ ]:
#ranging_list